In [1]:
# Import Necessary Libraries
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython

from scipy.linalg import block_diag
from scipy import sparse

from qpsolvers import Problem, solve_problem

import quadrotor


/home/jithin/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
#initialisation
m=quadrotor.MASS      # 0.5
r=quadrotor.LENGTH    # 0.15
Ine=quadrotor.INERTIA # 0.1
dt=quadrotor.DT       # 0.04
xdim=quadrotor.DIM_STATE
udim=quadrotor.DIM_CONTROL
grav=quadrotor.GRAVITY_CONSTANT
# iterations = 2 # number of steps
# nvars = iterations*udim + (iterations+1)*xdim # total number of variables


In [3]:
def C_d_eqconst_function(Xbar: np.ndarray,x_init: np.ndarray,iterations, xdim=6, udim = 2, debug =False):
    
    
    indiv_state_size = xdim + udim    # 8
    
    C =np.zeros([xdim*iterations,(indiv_state_size)*iterations])
    C[0:xdim,0:xdim] = np.eye(xdim)
    
    
    
    d = np.zeros([xdim*iterations,1])
    d[0:xdim, 0] = x_init[:,0] - Xbar[0:xdim,0]
    
    
    for i in range(iterations-1):
        
        # Unpacking from Xbar:
        px_curr    = Xbar[i * (indiv_state_size),0]
        vx_curr    = Xbar[i * (indiv_state_size)+1,0]
        py_curr    = Xbar[i * (indiv_state_size)+2,0]
        vy_curr    = Xbar[i * (indiv_state_size)+3,0]
        theta_curr = Xbar[i * (indiv_state_size)+4,0]
        omega_curr = Xbar[i * (indiv_state_size)+5,0]
        u1_curr    = Xbar[i * (indiv_state_size)+6,0]
        u2_curr    = Xbar[i * (indiv_state_size)+7,0]

        px_next    = Xbar[(i+1) * (indiv_state_size),0]
        vx_next    = Xbar[(i+1) * (indiv_state_size)+1,0]
        py_next    = Xbar[(i+1) * (indiv_state_size)+2,0]
        vy_next    = Xbar[(i+1) * (indiv_state_size)+3,0]
        theta_next = Xbar[(i+1) * (indiv_state_size)+4,0]
        omega_next = Xbar[(i+1) * (indiv_state_size)+5,0]


        # Set our A_alpha, A_beta, B_alpha, B_beta, B_gamma
        A_alpha = -((u1_curr + u2_curr)*dt*np.cos(theta_curr))/m
        A_beta  = -((u1_curr + u2_curr)*dt*np.sin(theta_curr))/m

        B_alpha = -(dt/m)*np.sin(theta_curr)
        B_beta  =  (dt/m)*np.cos(theta_curr)
        B_gamma =  (r*dt)/Ine

        A = np.array([
                     [1.0, dt, 0.0, 0.0, 0.0, 0.0],
                     [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                     [0.0, 0.0, 1.0, dt, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 0.0, 1.0, dt],
                     [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
                     ])

        A[1,4]=A_alpha
        A[3,4]=A_beta

        B = np.zeros([6,2])

        B[1, 0] =  B_alpha
        B[1, 1] =  B_alpha
        B[3, 0] =  B_beta
        B[3, 1] =  B_beta
        B[5, 0] =  B_gamma
        B[5, 1] = -B_gamma

        # Construct C matrix
        C[(xdim*(i+1)):xdim*(i+2) , (indiv_state_size)*i:((indiv_state_size)*i+6)]=A
        C[(xdim*(i+1)):xdim*(i+2) , ((indiv_state_size)*i+6):((indiv_state_size)*i+8)]=B    
        C[(xdim*(i+1)):xdim*(i+2) , ((indiv_state_size)*i+8):((indiv_state_size)*i+14)]=-np.eye(xdim)
        
        # Construct d matrix
        idx = (i + 1) * xdim
        d[idx + 0,0] = px_curr    + dt*vx_curr    -  px_next
        d[idx + 1,0] = vx_curr    - (dt*(u1_curr  + u2_curr)*np.sin(theta_curr))/m - vx_next 
        d[idx + 2,0] = py_curr    + dt*vy_curr    -  py_next
        d[idx + 3,0] = vy_curr    + (dt*(u1_curr  + u2_curr)*np.cos(theta_curr))/m - dt*grav -vy_next
        d[idx + 4,0] = theta_curr + dt*omega_curr - theta_next
        d[idx + 5,0] = omega_curr + B_gamma*(u1_curr-u2_curr) - omega_next

    if debug:
        for i in range(iterations-1):

            dim=([xdim*iterations,(xdim+udim)*iterations])
            second_interval=((xdim+udim)*i+14)
            first_interval=((xdim+udim)*i+8)
            last_indexing_interval=np.array([first_interval,second_interval])

        print(f"The last indexing interval of the equality matrix C: {last_indexing_interval}, the second index should be less than length of Xbar: {dim[1]} - 2")
        print(f"Current shape of C matrix = {C.shape}")
        
    #print("PAss")
    
    return C.astype(np.float64), d.astype(np.float64)

In [4]:
iterations=100
Xbar=np.zeros([(xdim+udim)*iterations,1])
x_init=np.zeros([6,1])
x_init.shape
d=C_d_eqconst_function(Xbar,x_init,iterations)[0]
d.shape

(600, 800)

In [5]:
def G_g_function(Xbar: np.ndarray,x_init: np.ndarray,iterations, xdim=6, udim = 2, debug =False):
    
    # G matrix construction  
    Q = np.diag([31.25, 0.03125, 31.25, 0.03125,
                    3.125, 0.03125])
    R = np.diag([0.75, 0.75])

    Matrix=np.block([
                    [Q, np.zeros((6, 2))],
                    [np.zeros((2, 6)), R]
                    ])

    Unit1 = [Matrix]*iterations
    G     = (block_diag(*Unit1)).astype(np.float64) 
    
    # g matrix construction
    g = np.zeros([(xdim + udim)*iterations,1])
    cluster = iterations / 5
    
    for i in range(iterations):
    
        if i <= cluster:
              x_des = np.array([0, 0, 0, 0, 0, 0])
              
        elif cluster <= i <= cluster*2:
            x_des = np.array([1, 0, 1, 0, (np.pi / 2), 0])
            
        elif cluster*2 < i <= cluster*3:
            x_des = np.array([0, 0, 6, 0, (np.pi), 0]) 
            
        elif cluster*3 < i <= cluster*4:
            x_des = np.array([-1, 0, 1, 0, 3 * (np.pi / 2), 0]) 
        elif cluster*4 < i <= iterations:
            x_des = np.array([0, 0, 0, 0, 2 * (np.pi), 0]) 
             
        XdesQ=(x_des@Q).reshape(-1,1) #shape(6,1)
        g[i*(xdim+udim):i*(xdim+udim)+6] = -XdesQ # -ve of XdesQ
        
    
    return G.astype(np.float64), g.astype(np.float64)

In [6]:
G,g = G_g_function(Xbar,x_init,iterations, xdim=6, udim = 2, debug =False)
print(f"shape shoud be equal to ({(xdim+udim)*iterations},{(xdim+udim)*iterations}) = > {G.shape}")
print(f"shape shoud be equal to ({(xdim+udim)*iterations},1) = > {g.shape}")

shape shoud be equal to (800,800) = > (800, 800)
shape shoud be equal to (800,1) = > (800, 1)


In [7]:
def cost_function(Xbar: np.ndarray, x_init: np.ndarray, iterations, xdim=6, udim = 2, debug =False):
    
    G,g  = G_g_function(Xbar ,x_init ,iterations, xdim, udim, debug)
    #print(f"G shape = {G.shape}, g shape = {g.shape}, Xbar shape = {Xbar.shape}")
    cost = ((0.5 * ((Xbar.T @ G) @ Xbar) + (2*g.T) @ Xbar)[0, 0]).astype(np.float64)
    #print(f"returns the scalar value of cost from the 2D array: {cost}")
    
    return cost

In [8]:
a=cost_function(Xbar, x_init, iterations, xdim=6, udim = 2, debug =False)
print(a)

0.0


In [9]:
def gradient_cost(Xbar: np.ndarray, x_init: np.ndarray, iterations, xdim=6, udim = 2, debug =False):
    
    G,g  = G_g_function(Xbar ,x_init ,iterations, xdim, udim, debug)
    #print(f"G shape = {G.shape}, g shape = {g.shape}, Xbar shape = {Xbar.shape}")
    costgrad = (G @ Xbar + g)
    
    return costgrad.astype(np.float64)

In [10]:
cg=gradient_cost(Xbar, x_init, iterations)
print(cg.shape)

(800, 1)


In [11]:
def Hessian_cost(Xbar: np.ndarray, x_init: np.ndarray, iterations, xdim=6, udim = 2, debug =False):
    
    G,_  = G_g_function(Xbar ,x_init ,iterations, xdim, udim, debug)
    #print(f"G shape = Hessian = {G.shape}")
    
    
    return G.astype(np.float64)

In [12]:
Hes=Hessian_cost(Xbar, x_init, iterations)
print(Hes.shape)

(800, 800)


In [13]:
def H_h_functions(Xbar: np.ndarray, iterations, debug=False):
    
    # H matrix construction
    Matrix=np.block([
                    [0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, -1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 0, 0, 0, -1],
                    [0, 0, -1, 0, 0, 0, 0, 0]
                    ])
    
    Unit1 = [Matrix]*iterations
    H     = (block_diag(*Unit1)).astype(np.float64) 
    
    # h matrix construction
    h = np.zeros([5*iterations,1]) 
    for i in range(iterations):
        h[5*i  ] = 10 - Xbar[6+i*8]   # u1<10
        h[5*i+1] = Xbar[6+i*8]        # u1>0
        h[5*i+2] = 10 - Xbar[7+i*8]   # u2<10
        h[5*i+3] = Xbar[7+i*8]        # u2>0
        h[5*i+4] = Xbar[2+i*8]        # for Py constraint
    
    return H.astype(np.float64),h.astype(np.float64)

In [14]:
H,h = H_h_functions(Xbar, iterations)
    
print(h.shape)

(500, 1)


In [15]:
def tot_constraint_violation_eq_ineq(Xbar: np.ndarray,x_init,iterations,debug = False):
    
    constraint_violation_eq =np.sum(abs(C_d_eqconst_function(Xbar,x_init,iterations)[1]))
    constraint_violation_ineq = 0
    
    for i in range(iterations):
        if Xbar[6+i*8] > 10:
            constraint_violation_ineq += abs(10 - Xbar[6+i*8])       # if u1 > 10
        elif Xbar[6+i*8] <0:            
            constraint_violation_ineq += abs(Xbar[6+i*8])            # if u1 < 0

        if Xbar[7+i*8] > 10:            
            constraint_violation_ineq += abs(10 - Xbar[7+i*8])       # if u2 > 10
        elif Xbar[7+i*8] <0:                
            constraint_violation_ineq += abs(Xbar[7+i*8])            # if u2 < 0

        if Xbar[2+i*8] < 0:         
            constraint_violation_ineq += abs(Xbar[2+i*8])            # if py < 0
            
    if debug:
        print(
            f"Equality Violation = {constraint_violation_eq}   Inequality Violation = {constraint_violation_ineq}")
    
    return (constraint_violation_eq + constraint_violation_ineq).astype(np.float64)

In [16]:
#x_init = np.zeros([6,1])
#N=100
#Xbar = np.zeros([8*N,1])
a = tot_constraint_violation_eq_ineq(Xbar, x_init, iterations, debug = True)

Equality Violation = 38.8476   Inequality Violation = 0


In [17]:
def solve_KKT_eq_ineq_constr(Xbar: np.ndarray,x_init,iterations): #Need to correct <==================
    
   A,b = C_d_eqconst_function(Xbar,x_init,iterations)
   P = Hessian_cost(Xbar, x_init, iterations)
   q = gradient_cost(Xbar, x_init, iterations)
   H, h = H_h_functions(Xbar, iterations)
   problem = Problem(P=P,q=q,A=A,b=-b.flatten(),G=H,h=h.flatten())
   
   # print(f"Xbar shape = {Xbar.shape}")
   # nvars = (iterations + 1) * xdim + iterations * udim 
   # print(f"nvars = (iterations + 1) * xdim + iterations * udim ")
   # print(f"nvars = ({iterations} + 1) * {xdim} + {iterations} * {udim} = {nvars}")
   # print(f"xdim * iteration ={xdim} * {iterations} = {xdim*iterations}")
   # print(f"C Shape = ({xdim} * {iterations},({iterations} + 1) * {xdim} + {iterations} * {udim})) = {C_Mat.shape}")
   # print(f"d Shape = {d_Mat.shape}")
   # print(f"G Shape = {H_mat.shape}")
   # print(f"grad Shape = {fgrad_Mat.shape}")
   # print(f"H Shape = {H.shape}")
   # print(f"h Shape = {h.shape}")
   
   solution = solve_problem(problem=problem,solver="cvxopt",verbose=False,initvals=None)
   return solution


In [19]:
x_guess=np.zeros(((xdim+udim)*100,1))
f_best=np.inf
c_best=np.inf
alpha=1
rho=0.05
x_init=np.zeros([6,1])

f_history=[]
c_history=[]
alpha_history=[]
iterations=100
tol = 1e-6

for i in range (500):
    res= solve_KKT_eq_ineq_constr(Xbar=x_guess,x_init=x_init,iterations=iterations)
    pk = res.x.reshape(-1, 1)
    
    while (cost_function(Xbar=(x_guess + alpha * pk),x_init=x_init,iterations= iterations) >= f_best and
         tot_constraint_violation_eq_ineq(Xbar=(x_guess + alpha * pk),x_init=x_init,iterations= iterations) > c_best):
          alpha = rho * alpha

    f_best = cost_function(Xbar=(x_guess + alpha * pk),x_init=x_init,iterations= iterations)
    c_best = tot_constraint_violation_eq_ineq(Xbar=(x_guess + alpha * pk),x_init=x_init,iterations= iterations)         
    alpha_history.append(alpha)
    c_history.append(c_best)
    f_history.append(f_best)
    print(f"At iteration {i} : Cost = {np.round(f_best, 5)},  Constraint Violation = {np.round(c_best, 5)}, \ Tolerance = {tol}  alpha = {alpha}")
    
    x_guess = x_guess + (alpha * pk)
    
    # px    =
    
    # theta = x_guess[0::3].T
    # omega = x_guess[1::3].T
    # controls = x_guess[2::3].T         
    if c_best < tol:
        break
    
# theta = x_guess[0::3].T
# omega = x_guess[1::3].T
# controls = x_guess[2::3].T
print(f"Total iterations needed = {i+1}")
print("DEBUG POINT")

At iteration 0 : Cost = -33914.47795,  Constraint Violation = [40.46837], \ Tolerance = 1e-06  alpha = 1
At iteration 1 : Cost = -38391.07407,  Constraint Violation = [29.33805], \ Tolerance = 1e-06  alpha = 1
At iteration 2 : Cost = -37639.4708,  Constraint Violation = [7.3937], \ Tolerance = 1e-06  alpha = 1
At iteration 3 : Cost = -37580.02171,  Constraint Violation = [3.69964], \ Tolerance = 1e-06  alpha = 1
At iteration 4 : Cost = -37656.83362,  Constraint Violation = [1.16347], \ Tolerance = 1e-06  alpha = 1
At iteration 5 : Cost = -37579.26657,  Constraint Violation = [0.68275], \ Tolerance = 1e-06  alpha = 1
At iteration 6 : Cost = -37610.66427,  Constraint Violation = [0.47208], \ Tolerance = 1e-06  alpha = 1
At iteration 7 : Cost = -37593.21213,  Constraint Violation = [0.3643], \ Tolerance = 1e-06  alpha = 1
At iteration 8 : Cost = -37595.9405,  Constraint Violation = [0.29741], \ Tolerance = 1e-06  alpha = 1
At iteration 9 : Cost = -37593.94705,  Constraint Violation = [0.2

In [29]:
 # Reshape y_guess into state and control trajectories
y_guess = x_guess.reshape(-1, 8)
x = y_guess[:, :quadrotor.DIM_STATE].T
u = y_guess[:, quadrotor.DIM_STATE:].T

quadrotor.animate_robot(x, u)

Your browser does not support the video tag.